# Phase-plane plot analysis

This notebook shows how to create a phase-plane plot of action potential. To read the full tutorial, please visit [Patch-clamp data analysis in Python: action potentials](https://spikesandbursts.wordpress.com/2022/05/03/patch-clamp-analysis-python-action-potentials/) of the [Spikes and Bursts](https://spikesandbursts.wordpress.com/) blog.

Differentials with numpy: https://numpy.org/doc/stable/reference/generated/numpy.diff.html

How to define a create a function in Python: https://swcarpentry.github.io/python-novice-inflammation/08-func/index.html

# Import the libraries

In [ ]:
#Import the packages and load the data
import pyabf
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
# import matplotlib as mpl

# Create the paths

In [ ]:
notebook_name = 'action_potentials_phase_plot'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data':  f'{data_path}/Data',
         'processed_data': f'{data_path}/Processed_data/{notebook_name}',
         'analysis': f'{data_path}/Analysis/{notebook_name}'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Load the data

Example data for this notebooks is the file **pfc_pvalb_aps_02.abf**.

In [ ]:
# ABF file/s
filename = "pfc_pvalb_aps_02"

data_path = f"{paths['data']}/{filename}.abf" 
abf = pyabf.ABF(data_path)
print(abf)

# Quick plot to see the trace/s
plt.figure(figsize=(8,4))

for sweepNumber in abf.sweepList:
    abf.setSweep(sweepNumber)
    plt.plot(abf.sweepX, abf.sweepY)
    plt.ylabel(abf.sweepLabelY)
    plt.xlabel(abf.sweepLabelX)

plt.show()

# Function

Read more about [creating functions](https://swcarpentry.github.io/python-novice-inflammation/08-func.html) from software carpentry.

* [numpy.diff](https://numpy.org/doc/stable/reference/generated/numpy.diff.html) documentation.

In [ ]:
# Function to calculate dv/dt
def dv_dt (abf, sweep, sampling_rate, half_window_ms):
    
    # Select the trace in your recording
    sweep = abf.setSweep(sweepNumber=sweep, channel=0)
    
    # Identify the action potential with highest dV
    dv = np.diff(abf.sweepY)
    dv_max_idx = np.argmax(dv)  # index of max dV/dt
    
    # Set the time window for the selected action potential
    sampling_rate = sampling_rate
    half_window_ms = half_window_ms
    half_window = (half_window_ms * sampling_rate)//1000
    global t_window
    t_window = abf.sweepX[dv_max_idx-half_window:dv_max_idx+half_window]
    v_window = abf.sweepY[dv_max_idx-half_window:dv_max_idx+half_window]
    
    # Calculate dv_dt for the selected action potential
    dv_ap = np.diff(v_window)
    dt_ap = np.diff(t_window*1000)
    dv_dt = (dv_ap)/(dt_ap)
    dv_dt_max = np.amax(dv_dt)
    # Remove the first v value to match the dv/dt array which is n-1
    v_ap_array = np.delete(v_window, 1)
    
    # Return the selected outputs from the function
    return {'voltage': v_window, 
            'dv_dt': dv_dt, 
            'dv_dt_max': dv_dt_max, 
            'dv_dt_max_idx': dv_max_idx, 
            'time': t_window,
            'voltage_array': v_ap_array}

# Results

In [ ]:
# Sweep of the first action potential
sweep_number = 4

# Current step related to sweep_number
abf.setSweep(sweep_number)
current_step = np.max(abf.sweepC)

# Sampling rate
fs = int(abf.dataPointsPerMs * 1000)

# Function inputs:  abf file, sweep number, sampling rate, half-window in ms
dv_dt_results = dv_dt(abf, sweep_number, fs, 6)

# Table with the voltage and dV/dt values
table = pd.DataFrame(columns = ['voltage', 'dv_dt'])  
table.voltage = pd.DataFrame(dv_dt_results['voltage'])
table.dv_dt = pd.DataFrame(dv_dt_results['dv_dt'])

# Graphs
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4))

# All sweeps
ax1.set_title('Trace')
abf.setSweep(sweep_number)
ax1.plot(abf.sweepX, abf.sweepY)
ax1.set_ylabel("V (mV)") 
ax1.set_xlabel("t (s)")
ax1.set_xlim(0.2, 0.8)

# Plot a red dot at the detected AP
dv_max_idx = dv_dt_results['dv_dt_max_idx']
ax1.plot(abf.sweepX[dv_max_idx], abf.sweepY[dv_max_idx], 'ro', markersize=6)

# Action potential
ax2.set_title('Action potential')
ax2.plot(dv_dt_results['time'], dv_dt_results['voltage'])
ax2.set_ylabel("V (mV)") 
ax2.set_xlabel("t (s)")

# Phase plot
ax3.set_title('Phase-plane plot')
ax3.plot(dv_dt_results['voltage_array'], dv_dt_results['dv_dt'])
ax3.set_ylabel("dV/dt (mV/ms)") 
ax3.set_xlabel("V (mV)") 

fig.tight_layout()

# Print the maximum dv/dt value, graph and table
print("dv/dt max =", dv_dt_results['dv_dt_max'])
print("Current step =", current_step)

# Save the plot and the table
fig.savefig(f"{paths['analysis']}/{filename}_{current_step}pA_phase_plot.png", dpi=300)
table.to_csv(f"{paths['analysis']}/{filename}_{current_step}pA_phase_plot.csv", index=False)

# Show the plots and table
plt.show()
table